In [4]:
# push!(LOAD_PATH, raw"C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\camera-calibration") # Point to Seth's repo location
# %matplotlib notebook
import CSV, DataFrames
import PyPlot as plt
import Glob
import Statistics#, NaNStatistics
import Geodesy
import Pkg
import Dates
import LaTeXStrings
import TiffImages
import FixedPointNumbers
import HDF5
import IRQIV
plt.pygui(true) # sets plots to popup instead of inline

true

In [ ]:
# Select & filter GPS data from IntertialSense EVB-2
plt.close()
GPSpath = raw"C:\Users\evanh\Box\Cornell\CACO\2022\CACO_GPS"
folders = readdir(GPSpath)
GCPxyz_us = Any[]
GCPlla_us = Any[]
GCPxyz_fix_ds = Any[]
fixpts = Any[]
gcp6a = Any[]
gcpo1a = Any[]
GCPnames = Any[]
j = 0
pltdim = 1 # dimension to plot
for (i,folder) in enumerate(folders)
    fullfile = GPSpath * "\\" * folder
    filename = readdir(Glob.glob"*POS.csv", fullfile)
    if occursin("US_FOV1", fullfile) == true || occursin("_DS_", fullfile) == true # Not using FOV1, focused on US only
        continue
    end
    GCP = CSV.read(filename, DataFrames.DataFrame)
    pos = GCP[:,[9, 10, 11, 16]] # lla 1-3 + status (code 0x00000C00 = RTK_FIX according to pg. 138: https://docs.inertialsense.com/user-manual/reference/user_manual_pdf/InertialSenseDocs.pdf)
    posfix = filter(row->any(occursin.("C", row.status[end-2:end-2])), pos) # from https://discourse.julialang.org/t/better-way-to-filter-strings-from-dataframe/88012
    if length(posfix.status) == 0 # No RTK_FIX (filtered out)
        posnf = pos
        if occursin("FOV2_GCPo2", fullfile) == true # Filtering jump
            deleteat!(posnf, 103:size(posnf,1))
            replace!(posnf[!,"lla[0]"] .= maximum(posnf[!,"lla[0]"]))
#             deleteat!(posnf, 1:140)
#             deleteat!(posnf, 1:1165)
#             dump(posnf[:,1])
#             show(posnf[:,"lla[0]"])
        end
        if occursin("FOV2_GCP1", fullfile) == true # Acceleration filter
            for k = 1:(size(posnf,1)-1)
                k = k+1
                if posnf[k-1,3] - posnf[k,3] > .5
                    posnf[k,:] = posnf[k-1,:]
                end
            end
#             deleteat!(posnf, 1660:2771)
            replace!(posnf[!,"lla[1]"] .= Statistics.median(posnf[!,"lla[1]"]))
        end
        if occursin("FOV2_GCP3", fullfile) == true # Filtering
            deleteat!(posnf, 2329:size(posnf,1))
            replace!(posnf[!,"lla[1]"] .= maximum(posnf[!,"lla[1]"]))
#             replace!(posnf[!,"lla[0]"] .= maximum(posnf[!,"lla[0]"]))
        end
        if occursin("FOV2_GCP5", fullfile) == true # Filtering
            continue
#             deleteat!(posnf, 2323:size(posnf,1))
#             deleteat!(posnf, 1:745)
#             deleteat!(posnf, 2324:size(posnf,1))
#             deleteat!(posnf, 1:2321)
#             replace!(posnf[!,"lla[0]"] .= Statistics.mean(posnf[!,"lla[0]"][250:350]))
            replace!(posnf[!,"lla[1]"] .= minimum(posnf[!,"lla[1]"]))
        end
#         dump(last(fullfile, 8))
        plt.plot(posnf[:,pltdim], label = last(fullfile, 8))
        pos = posnf
    else
        push!(fixpts, length(posfix.status), last(fullfile, 9))
        if occursin("DS_GCPb6", fullfile) == true # Filtering jump
#             deleteat!(posfix, 56:140)
        end
        if occursin("DS_GCPb3", fullfile) == true # Filtering jump - roughly half high and half low (average?)
            deleteat!(posfix, 227:249)
        end
        if occursin("DS_GCPo2", fullfile)
            continue
        end
        j = j+1
        pos = posfix
        plt.plot(posfix[:,pltdim], label = last(fullfile, 8))
    end
#     # visualize spread before taking mean, or look at SDs
    avgLLA = Statistics.mean.(eachcol(pos[:,1:3]))
#     plt.scatter(x = avgLLA[2], y = avgLLA[1], label = last(fullfile, 8))#, label = last(fullfile, 8))
#     if occursin("FOV2_GCP6", fullfile) == true
#         push!(gcp6a, avgLLA[3]) # save elevation for US GCP (on deck) to guess US camera elevation
#     end
#     if occursin("DS_GCP", fullfile) == true
#         push!(gcpo1a, avgLLA[3]) # save elevation for DS GCP (lowest bolt) to guess DS camera elevation
#     end
    GCP_LLA = Geodesy.LLA(avgLLA[1], avgLLA[2], avgLLA[3])
#     dump(filename)
#     dump(GCP_LLA)
    GCP_UTM = Geodesy.UTMZ(GCP_LLA, Geodesy.wgs84) # Inertialsense uses WGS84 - see manual link above
#     if occursin("US_FOV2", fullfile) == true
    push!(GCPxyz_us, GCP_UTM)
    push!(GCPlla_us, GCP_LLA)
#     elseif occursin("_DS_", fullfile) == true 
#         push!(GCPxyz_fix_ds, GCP_UTM)
#     else continue
#     end
    push!(GCPnames, last(fullfile, 9))
    dump(last(fullfile, 9))
    dump(GCP_LLA)
end
plt.legend()
# dump(GCPxyz_us)
# GCPnames
# fixpts
# DS GCPs b7 & b6 are over 1m apart in elevation on figure, actually only 2.5 in apart on fence??
# US GCP 4 over 2m higher than all others??

In [5]:
# Camera location approximation:
camll_us = [41.931168, -70.064190] # Guess from google maps
camll_ds = [41.931099, -70.064524] # ''
ft_to_m = 0.3048
camelev_us = sqrt(9^2-2^2)*ft_to_m # Have measurement of camera to deck (?) for US side
# camelev_ds = gcpo1a[1] + 6.5*ft_to_m # Guess DS camera offset from board GCP elev based on photos
cam_LLA_us = Geodesy.LLA(camll_us[1], camll_us[2], camelev_us)
cam_UTM_us = Geodesy.UTMZ(cam_LLA_us, Geodesy.wgs84)
# cam_LLA_ds = Geodesy.LLA(camll_ds[1], camll_ds[2], camelev_ds)
# cam_UTM_ds = Geodesy.UTMZ(cam_LLA_ds, Geodesy.wgs84)

# Camera angle approximation:
fl = 1214.2857142857144 # In pixels - from Seth
azimuth_us = deg2rad(5) # Guess from google maps
tilt_us = atan(70/62) # Approx. from photo
roll_us = atan(20/85) # ''
# azimuth_ds = 315 * (pi/180) # Guess from google maps
# tilt_ds = atan(120/34) # Approx. from photo
# roll_ds = atan(7/50) # ''

ECPs_us = IRQIV.ExtrinsicCalibrationParameters(azimuth_us, tilt_us, roll_us, fl, cam_UTM_us.x, cam_UTM_us.y, cam_UTM_us.z)
# ECPs_ds = ECC.ExtrinsicCalibrationParameters(azimuth_ds, tilt_ds, roll_ds, fl, cam_UTM_ds.x, cam_UTM_ds.y, cam_UTM_ds.z)
dump(ECPs_us)
# dump(ECPs_ds)

IRQIV.ExtrinsicCameraCalibration.ExtrinsicCalibrationParameters
  φ: Float64 0.08726646259971647
  τ: Float64 0.8459301831795555
  σ: Float64 0.23109066719589708
  f: Float64 1214.2857142857144
  x_c: Float64 411770.8466259552
  y_c: Float64 4.6426814979640795e6
  z_c: Float64 2.674609145277119


In [ ]:
# Guesses of GCP elevation, with bridge deck as z=0:
# Posts are 35" high off bridge deck based on photos from ME, 0.0254 m/in
us_bo1_zg = [0] # Estimated on deck
us_bo2_zg = [0] # "
us_GCP1_zg = [(35-4)*0.0254] # Tape approx. 4" below top of fence
us_GCP2_zg = [(17.5+6)*0.0254] # Top of first rung is 17.5" from deck, + 6" to center of 12" GCP
us_GCP3_zg = [(17.5+6)*0.0254] # "
us_GCP4_zg = [(35-4)*0.0254] # Tape approx. 4" below top of fence
us_GCP5_zg = [0] # On deck
us_GCP6_zg = [2*0.0254] # On wood cover ~2" above deck
GCPzg_us = [us_bo1_zg, us_bo2_zg, us_GCP2_zg, us_GCP3_zg, us_GCP4_zg, us_GCP1_zg, #us_GCP5_zg, 
    us_GCP6_zg]

In [ ]:
# View selected images of GCPs - hit enter in readline to advance figure
plt.close()
IMGpath = raw"C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\GCP_photos"
folders = readdir(IMGpath)
for (i,folder) in enumerate(folders)
    fullfile = IMGpath * "\\" * folder
    image = plt.imread(fullfile)
#     dump(size(image))
    plt.imshow(image; extent = [1,1344,784,1])
    plt.title(last(fullfile, 9))
    readline()
end
# DS board o2 out of frame

In [ ]:
# Finding GCPs in uv coordinates
# upper left is 0,0 viewing original .ats files in ResearchIR
us_bo1_uv = [475, 67]
us_bo2_uv = [1034, 570]
us_GCP1_uv = [279, 473]
us_GCP2_uv = [164, 248]
us_GCP3_uv = [45, 83]
us_GCP4_uv = [14, 15]
us_GCP5_uv = [235, 612]
us_GCP6_uv = [40, 741]
# Downstream - needs correction
# ds_board_o1_uv = [350.2, 40.4]
# ds_b1_uv = [725.0, 257.6]
# ds_b2_uv = [719.9, 268.9]
# ds_b3_uv = [694.3, 318.1]
# ds_b5_uv = [667.6, 356.3]
# ds_b6_uv = [731.3, 15.2]
# ds_b7_uv = [726.5, 43.4]
GCPuv_us = [us_bo1_uv, us_bo2_uv, us_GCP2_uv, us_GCP3_uv, us_GCP4_uv, us_GCP1_uv, #us_GCP5_uv, 
    us_GCP6_uv]
# GCPuv_fix_ds = [ds_board_o1_uv, ds_b7_uv, ds_b6_uv, ds_b3_uv]

# Finding uv coords of board tape
bo1_uv_us = [us_bo1_uv,
[456,101],
[437,136],
[415,172],
[395,208],
[373,245],
[352,283],
[330,321],
[306,360],
[285,400]]
bo2_uv_us = [us_bo2_uv,
[976,603],
[916,635],
[858,668],
[799,697],
[743,728],
[688,758],
[634,784]]

In [ ]:
# Adding in board tape as additional GCPs 
plt.close()
tapeL = 0.5 * ft_to_m # tape distance is 6"
bo1_ant_us = [GCPxyz_us[1].x, GCPxyz_us[1].y]
bo1_ang_us = atan((504-275)/(433-38))+azimuth_us # atan(x/y) = degrees off of north + azimuth = estimate
bo1_tape_us = fill(0.0, 10, 2) # 10 pieces of tape on board, x&y
bo1_tape_us[1,:] = bo1_ant_us
for i = 1:9
    j = i+1
    bo1_tape_us[j,1] = bo1_ant_us[1] - (sin(bo1_ang_us)*tapeL*i) # multiply by index: # of 6" increments
    bo1_tape_us[j,2] = bo1_ant_us[2] - (cos(bo1_ang_us)*tapeL*i)
end
dump(bo1_tape_us)
bo2_ant_us = [GCPxyz_us[2].x, GCPxyz_us[2].y]
bo2_ang_us = atan((1107-667)/(781-535))+azimuth_us # atan(x/y) = degrees off of north + azimuth = estimate
bo2_tape_us = fill(0.0, 8, 2) # Only 8 pieces of tape visible
bo2_tape_us[1,:] = bo2_ant_us
for i = 1:7
    j = i+1
    bo2_tape_us[j,1] = bo2_ant_us[1] - (sin(bo2_ang_us)*tapeL*i) # multiply by index: # of 6" increments
    bo2_tape_us[j,2] = bo2_ant_us[2] - (cos(bo2_ang_us)*tapeL*i)
end
dump(bo2_tape_us)
plt.scatter(bo1_tape_us[:,1], bo1_tape_us[:,2])
plt.scatter(bo2_tape_us[:,1], bo2_tape_us[:,2])
bo1_tape_us[:,2]
sqrt((GCPxyz_us[2].x-GCPxyz_us[1].x)^2+(GCPxyz_us[2].y-GCPxyz_us[1].y)^2)

In [ ]:
thermpath = raw"C:\Users\evanh\Box\Cornell\CACO\2022\Thermistor\P6_CACO_July_2022.txt"
therm = CSV.read(thermpath, delim = ',', skipto = 48, DataFrames.DataFrame)
thermP = therm[:,2]
plt.plot(thermP)
# 360 lines per hour, 8640 lines per day
us_therm_z = (sqrt(9^2-2^2) + 1/3 + 7) * ft_to_m
ds_therm_z = 209/12*ft_to_m
# therm[9230,:] - thermistor started @ 6:30 on 7/13, moved downstream @ 8:12 am on 7/14
thermZ = us_therm_z.-thermP[1:9230]
thermZ = append!(thermZ, ds_therm_z.-thermP[9231:end])

plt.close("all")
thermT = string.(therm[:,4])
plt.plot(thermT[6302:6302+8638], thermP[6302:6302+8638])
plt.xticks(1:1200:length(thermZ[6302:6302+8638])#, rotation = 45
)
plt.xlabel("Time - July 14, 2022")
plt.ylabel("Thermistor pressure head (m)")
plt.rc("font",size=24)

# thermZf = open("therm_z_us.txt","w")
# show(thermZf, thermP[871:9230]) 
# close(thermZf)
# thermZread = open("therm_us_readme.txt", "w")
# write(thermZread, "Vertical distance of water surface to camera (m) from 8:55:00 on 7/13/22 to 8:08:10 on 7/14/22; \n")
# write(thermZread, "Adjusted by vertical offset from camera to thermistor (4.91m) with thermistor head subtracted; \n")
# write(thermZread, "Adjusted by vertical offset from camera to thermistor (4.91m) with thermistor head subtracted; \n")
# write(thermZread, "Indices corresponding to provided .ATS files: \n")
# write(thermZread, "003349: [6621:6660] \n")
# write(thermZread, "003359: [6988:7008] \n")
# write(thermZread, "003368: [7354:7374] \n")
# write(thermZread, "003377: [7708:7728] \n")
# write(thermZread, "003386: [8068:8088] \n")
# close(thermZread)

In [ ]:
# plt.plot(thermZ[871:9230])
plt.plot(thermP[871:9230])
therm[8068+871:8088+871,:]
# plt.plot(thermP[6621+871:8088+871])

In [ ]:
# therm[871,:] # 8:55 7/13 - therm in water US
# therm[9230,:] # 8:08:10 7/14 - last in-water reading US
# using Pkg
# Pkg.activate(expanduser("~/.julia/dev/IRQIV/"))
# include("CameraCalibrationHelpers.jl")
# Pkg.activate(raw"C:\Users\evanh\.julia\dev\IRQIV.jl\examples\CameraCalibrationHelpers.jl") # path to IRQIV directory
# TrueGCPxyz = [GCPxyz_us.x GCPxyz_us[all].y GCPzg_us[all]]
GCPxyz_us[:].x

In [8]:
ι = IRQIV.Camera(FLIR_SC8303_17mm)

UndefVarError: UndefVarError: `FLIR_SC8303_17mm` not defined

In [ ]:
ι = IRQIV.IntrinsicCalibrationParameters(FLIR_SC8303_17mm)

# UTM coordinates of GCPs:
TrueGCPxyz = [GCPxyz_us[:,1] GCPxyz_us[:,2] GCPzg_us]

# Pixel coordinates of GCPs, before accounting for radial distortion:
DistortedImageGCPuv  = [ 1295.0  113.4
                         1244.3  725.3
                         1265.6  757.45
                         1335.0  138.3
                         115.7  471.5
                         137.6  272.5
                         173.6   57.1
                         82.4  761.0]

## Remove radial distortion from GCP target coordinates:
# 
# (the hcat other things on this line are just to convert the output from a
# Tuple to an Array)
TrueGCPuv = hcat(RemoveRadialDistortion(DistortedImageGCPuv[:,1], DistortedImageGCPuv[:,2], Camera)...)

# Camera position initial guess:
Camera_x = 624165.196
Camera_y = 4243306.056
Camera_z = 24.24 - 0.4          # antenna was placed above camera, 0.4m offset
#
# estimated camera orientation angles:
φ =  50                         # azimuth
τ = 215                         # tilt
σ =  90                         # roll
#
f = -1200 # nominal_focal_length_px (nominal lens focal length / pixel pitch)

# Initial guess of camera's extrinsic parameters:
χ = ExtrinsicCalibrationParameters(
    deg2rad(φ),
    deg2rad(τ),
    deg2rad(σ),
    f,
    Camera_x,  # x_c
    Camera_y,  # y_c
    Camera_z) # z_c


##############################################################################
plt.close("all")
GraphicallyExploreInitialCalibrationGuess(TrueGCPuv, TrueGCPxyz, χ, ι)# ,
                                          # frame=fill(NaN, ι.v₀*2 - 1 |> Int, ι.u₀*2 - 1 |> Int))
##############################################################################

##############################################################################
# Optionally, load an IR image and plot it in the background:
FLIR_filename = joinpath(raw"D:\CNRD_IR",
                         "Rec-DeFreesLab_-003349.ats")
IRimages = LoadATSImageSequence(FLIR_filename, 1, 2017)
# 
plt.close("all")
GraphicallyExploreInitialCalibrationGuess(TrueGCPuv, TrueGCPxyz, χ, ι, IRimages[1])# ,
##############################################################################


In [ ]:
###############################################################################
# 
# IRQIV_MWE.jl in /home/seth/.julia/dev/IRQIV/test/
# 
# Created: 2023-04-24 16:55:53 by seth@SASthinkpad
# last changed Time-stamp: <2023-04-26 15:08:17 seth>
# 
###############################################################################

###############################################################################
Pkg.activate(expanduser("~/.julia/dev/IRQIV/")) # path to IRQIV directory
using IRQIV
###############################################################################


###############################################################################
# 
### Select a dataset to operate on:
#
# Sutter Slough:
FLIR_filename = joinpath(raw"D:\CNRD_IR",
                         "Rec-DeFreesLab_-003318.ats") # 3349
ats_year = 2022
@assert isfile(FLIR_filename)
###############################################################################

###############################################################################
const M=64 # subwindow dims - horiz, N = vert.
const N=64
const rhoI = 20 # how far away to search (+/-) - i=horiz, j= vert
const rhoJ = 20 # Set these to visual estimates - plot one pair
MQD_parameters = (M=M, N=N, rhoI=rhoI, rhoJ=rhoJ)
###############################################################################


###############################################################################
DtFrames = 1                    # Δt, frames (ATS 3116 is at 10Hz)
data_output_rate_frames = 150    # difference between I1 sequence, frames - time sep of output
nImagePairs = 20 # can set to inf - all frames in file
# Can test on 10 pairs & look at histogram of displacements
###############################################################################


###############################################################################
# Load IR images:
I1inds = range(1, step=data_output_rate_frames, length=nImagePairs)
I2inds = I1inds .+ DtFrames
I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)
###############################################################################
# Remove image pairs when at least one image took place during NUC (calibration)
# mode:
if any(isnuc(I1images.Headers)) || any(isnuc(I2images.Headers))
    nuc_frames = isnuc(I1images.Headers) .| isnuc(I2images.Headers)
    I1inds = I1inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
    I2inds = I2inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
    I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
    I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)
end
###############################################################################

###############################################################################
subwincorners = CartesianIndices(( (N+rhoI):N:(size(I1images,1)-(N+rhoI+1)),
                                   (M+rhoJ):M:(size(I1images,2)-(M+rhoJ+1)) ))
# ranges determine N/M-overlap,  want to workshop sub-regions here
###############################################################################


###############################################################################
# Initialize MQD calculation:
MQD_variables = initialize_MQD(I1images,
                               I2images,
                               subwincorners,
                               N, M,
                               rhoI, rhoJ)                              
###############################################################################

###############################################################################
# Perform the MQD analysis:
subpix_shifts_i, subpix_shifts_j, performance_message = perform_MQD(
    I1images, I2images, subwincorners, MQD_variables, MQD_parameters);
###############################################################################
jmean = Statistics.mean(subpix_shifts_j[:,:,:], dims=3)
imean = Statistics.mean(subpix_shifts_i[:,:,:], dims=3)
jmean = dropdims(jmean, dims=3)
imean = dropdims(imean, dims=3)
meanmags = Statistics.mean(hypot.(jmean, imean))

i = [ x[1] for x in Tuple.(subwincorners)]
j = [ x[2] for x in Tuple.(subwincorners)]
plt.close("all")
fig,axs = plt.subplots()
im = axs.imshow(I1images[:,:,1], alpha=0.3, vmin=8700, vmax=11800)
q = axs.quiver(j,i, jmean, imean, angles="xy")
# ax.invert_yaxis()

In [ ]:
# Upstream IRQIV averaging
Pkg.activate(expanduser("~/.julia/dev/IRQIV/")) # path to IRQIV directory
using IRQIV


###############################################################################

###############################################################################
const M=64 # subwindow dims - horiz, N = vert.
const N=64
const rhoI = 40 # how far away to search (+/-) - i=horiz, j= vert
const rhoJ = 40 # Set these to visual estimates - plot one pair
MQD_parameters = (M=M, N=N, rhoI=rhoI, rhoJ=rhoJ)
###############################################################################


###############################################################################
DtFrames = 1                    # Δt, frames (ATS 3116 is at 10Hz)
data_output_rate_frames = 150    # difference between I1 sequence, frames - time sep of output
nImagePairs = 20 # can set to inf - all frames in file
# Can test on 10 pairs & look at histogram of displacements
###############################################################################


###############################################################################
# Load IR images:
I1inds = range(1, step=data_output_rate_frames, length=nImagePairs)
I2inds = I1inds .+ DtFrames

Nfiles = 37

meanmagsus = fill(NaN, Nfiles)
for f = 1:Nfiles
    file = f+3348
    FLIR_filename = joinpath(raw"D:\CNRD_IR",
                         "Rec-DeFreesLab_-00" * string.(file) * ".ats")
    ats_year = 2022
    @assert isfile(FLIR_filename)
    I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
    I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)

    if any(isnuc(I1images.Headers)) || any(isnuc(I2images.Headers))
        nuc_frames = isnuc(I1images.Headers) .| isnuc(I2images.Headers)
        I1inds = I1inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
        I2inds = I2inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
        I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
        I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)
    end

    subwincorners = CartesianIndices(( (N+rhoI):N:(size(I1images,1)-(N+rhoI+1)),
                                   (M+rhoJ):M:(size(I1images,2)-(M+rhoJ+1)) ))

    MQD_variables = initialize_MQD(I1images,
                                   I2images,
                                   subwincorners,
                                   N, M,
                                   rhoI, rhoJ)                              

    subpix_shifts_i, subpix_shifts_j, performance_message = perform_MQD(
        I1images, I2images, subwincorners, MQD_variables, MQD_parameters);
    ###############################################################################
    jmean = Statistics.mean(subpix_shifts_j[:,:,:], dims=3)
    imean = Statistics.mean(subpix_shifts_i[:,:,:], dims=3)
    jmean = dropdims(jmean, dims=3)
    imean = dropdims(imean, dims=3)
    meanmagsus[f] = Statistics.mean(hypot.(jmean, imean))

    i = [ x[1] for x in Tuple.(subwincorners)]
    j = [ x[2] for x in Tuple.(subwincorners)]
    
    plt.close("all")
    fig,ax = plt.subplots()
    im = ax.imshow(I1images[:,:,1], alpha=0.3, vmin=8700, vmax=11800)
    q = ax.quiver(j,i, jmean, imean, angles="xy")
    # ax.invert_yaxis()
    
    # Look at histogram of displacements
    fig, ax = plt.subplots()
    h1 = ax.hist(vec(subpix_shifts_i), 100, label="i", alpha=0.4);
    h2 = ax.hist(vec(subpix_shifts_j), 100, label="j", alpha=0.4);
    plt.legend()
    plt.show()
end

In [ ]:
# Downstream IRQIV averaging
Pkg.activate(expanduser("~/.julia/dev/IRQIV/")) # path to IRQIV directory
using IRQIV


###############################################################################

###############################################################################
const M=64 # subwindow dims - horiz, N = vert.
const N=64
const rhoI = 40 # how far away to search (+/-) - i=horiz, j= vert
const rhoJ = 40 # Set these to visual estimates - plot one pair
MQD_parameters = (M=M, N=N, rhoI=rhoI, rhoJ=rhoJ)
###############################################################################


###############################################################################
DtFrames = 1                    # Δt, frames (ATS 3116 is at 10Hz)
data_output_rate_frames = 150    # difference between I1 sequence, frames - time sep of output
nImagePairs = 20 # can set to inf - all frames in file
# Can test on 10 pairs & look at histogram of displacements
###############################################################################


###############################################################################
# Load IR images:
I1inds = range(1, step=data_output_rate_frames, length=nImagePairs)
I2inds = I1inds .+ DtFrames

Nfiles = 20

meanmagsds = fill(NaN, Nfiles)
for f = 1:Nfiles
    file = f+3485
    FLIR_filename = joinpath(raw"D:\CNRD_IR", # from RAID here - deeper file structure than NVME
                         "Rec-DeFreesLab_-00" * string.(file) * ".ats")
    ats_year = 2022
    @assert isfile(FLIR_filename)
    I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
    I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)

    if any(isnuc(I1images.Headers)) || any(isnuc(I2images.Headers))
        nuc_frames = isnuc(I1images.Headers) .| isnuc(I2images.Headers)
        I1inds = I1inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
        I2inds = I2inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
        I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
        I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)
    end

    subwincorners = CartesianIndices(( (N+rhoI):N:(size(I1images,1)-(N+rhoI+1)),
                                   (M+rhoJ):M:(size(I1images,2)-(M+rhoJ+1)) ))

    MQD_variables = initialize_MQD(I1images,
                                   I2images,
                                   subwincorners,
                                   N, M,
                                   rhoI, rhoJ)                              

    subpix_shifts_i, subpix_shifts_j, performance_message = perform_MQD(
        I1images, I2images, subwincorners, MQD_variables, MQD_parameters);
    ###############################################################################
    jmean = Statistics.mean(subpix_shifts_j[:,:,:], dims=3)
    imean = Statistics.mean(subpix_shifts_i[:,:,:], dims=3)
    jmean = dropdims(jmean, dims=3)
    imean = dropdims(imean, dims=3)
    meanmagsds[f] = Statistics.mean(hypot.(jmean, imean))

    i = [ x[1] for x in Tuple.(subwincorners)]
    j = [ x[2] for x in Tuple.(subwincorners)]
    
    plt.close("all")
    fig,ax = plt.subplots()
    im = ax.imshow(I1images[:,:,1], alpha=0.3, vmin=8700, vmax=11800)
    q = ax.quiver(j,i, jmean, imean, angles="xy")
    # ax.invert_yaxis()
    
    # Look at histogram of displacements
    fig, ax = plt.subplots()
    h1 = ax.hist(vec(subpix_shifts_i), 100, label="i", alpha=0.4);
    h2 = ax.hist(vec(subpix_shifts_j), 100, label="j", alpha=0.4);
    plt.legend()
    plt.show()
end

In [ ]:
meanmagsds = -meanmagsds
irqivTus = Array(Dates.Time(Dates.Hour(3), Dates.Minute(25)):Dates.Second(395):Dates.Time(Dates.Hour(7), Dates.Minute(23)))
irqivTus_str = string.(irqivTus)
irqivTds = Array(Dates.Time(Dates.Hour(20), Dates.Minute(58)):Dates.Second(400):Dates.Time(Dates.Hour(23), Dates.Minute(11)))
irqivTds_str = string.(irqivTds)

irqivDTus = Dates.DateTime[]
irqivDTds = Dates.DateTime[]
for i = 1:length(irqivTus)
    push!(irqivDTus, Dates.Date("07/14/2022", "m/d/y") + irqivTus[i]) # Times in EDT
end
for i = 1:length(irqivTds)
    push!(irqivDTds, Dates.Date("07/14/2022", "m/d/y") + irqivTds[i]) # Times in EDT
end

plt.close("all")
fig, (ax1, ax2) = plt.subplots(2)
ax1.plot(irqivTus_str, meanmagsus, linewidth = 2, color = "red")
ax2.plot(irqivTds_str, meanmagsds, linewidth = 2, color = "red")
ax1.set_xticks(1:6:length(meanmagsus), #rotation = 45
 )
ax2.set_xticks(1:4:length(meanmagsds), #rotation = 45
 )
plt.xlabel("Time - July 14, 2022")
plt.rc("font",size=24)
fig.text(0.04, 0.5, "IR-QIV mean vector magnitude (pix)", va="center", rotation="vertical")
ax1.legend(["Upstream outgoing tide"])
ax2.legend(["Downstream incoming tide"])

In [ ]:
# ax.invert_yaxis()
# angles = "xy"
# Statistics.mean(subpix_shifts_j[:,:,:], dim)

plt.close("all")
fig,ax = plt.subplots()
im = ax.imshow(I1images[:,:,1], alpha=0.3, vmin=8700, vmax=11800)
q = ax.quiver(j,i, jmean, imean, angles="xy")
# ax.plot(q)
# ax.invert_yaxis()

In [ ]:
extrema(I1images[1]).|>Int

In [ ]:
# For methane experiment
using Statistics
mean(I1images.Frames, dims=3)
# set indices to inf

In [ ]:
kb_model = CSV.read(raw"C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\Modeling\hr_heb_Verified_(m)_38834_dt360s_230418.csv", DataFrames.DataFrame)
mh_model = CSV.read(raw"C:\Users\evanh\Box\Cornell\CACO\2022\Analysis\Modeling\Delft3D_HR_Discharge.csv", DataFrames.DataFrame)
GMT_to_EDT = Dates.Hour(4)
kb_model.date_time = Dates.DateTime.(kb_model.date_time, "yyyy-mm-dd HH:MM:SS") - GMT_to_EDT # Convert to EDT here!
mh_model.Datetime = Dates.DateTime.(mh_model.Datetime, "m/d/y H:M") - GMT_to_EDT # Convert to ED
kb_model.time = Dates.Time.(kb_model.date_time)
mh_model.time = Dates.Time.(mh_model.Datetime)
# kb_time = Dates.Time[]
start = Dates.DateTime("07/14/2022 00:00:00", "m/d/y H:M:S")
stop = start + Dates.Day(1)
kb_filt = kb_model[(kb_model.date_time.>= start) .& (kb_model.date_time.<= stop), :]
mh_filt = mh_model[(mh_model.Datetime.>= start) .& (mh_model.Datetime.<= stop), :]
# timetix = 



# Plot results
plt.close("all")
fig, ax = plt.subplots()
ax.plot(kb_filt.date_time, kb_filt.Qtotal_m3s, linewidth = 2)
ax.plot(mh_filt.Datetime, mh_filt."Total_Discharge_m3/s", linewidth = 2)
ax.plot(irqivDTus, meanmagsus, linewidth = 2)
ax.plot(irqivDTds, meanmagsds, linewidth = 2)
plt.rc("font",size=24)
plt.ylabel(LaTeXStrings.LaTeXString("Modeled volumetric flux (m^3/s) \n IR-QIV mean vector magnitude (pix)"))
plt.xlabel("Hour of day (July 14, 2022)")
plt.legend(["Model: hydraulic mass balance", "Model: depth-averaged Delft3D", "IR-QIV: upstream outgoing tide", "IR-QIV: downstream incoming tide"], loc = "upper center")
plt.ylim([-20, 60])

In [ ]:
mh_filt.time[end]

In [ ]:
string(replace("test string", r"(test string)" => SubstitutionString("\\1\nline 2")))

In [ ]:
mh_filt.time
# Array(irqivTus)
# mh_filt.Datetime
# Dates.Time(kb_filt.date_time)

test = Dates.Date("07/14/2022", "m/d/y") + mh_model.time(:)
test
mh_filt.Datetime[1]
# Dates.Time(string(mh_model.time[1]),"H:M:S")
# mh_model.time[1]